In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy
from sympy import symbols, Poly, Eq, diff, lambdify, Piecewise, sinh, cosh, pi
from sympy.solvers import solve_undetermined_coeffs
from sympy.abc import x as var_x

%matplotlib inline

In [ ]:
class OdeHalfAnalyticalSolve:
    """ Only for temperature equation
        du/dt = a^2 d^2u/dt^2 """
    
    def __init__(self, k, a2, U_left, U_right, init_cond, t0, x0, step_t, step_x, T, L, degree=1, n_dots=4):
        
        # k          -    параметр рівняння                     type(k) == float
        # a2         -    параметр рівняння                     type(a2) == float
        # U_left     -    ліва гранична умова;                  type(U_left) == float
        # U_right    -    права гранична умова;                 type(U_right) == float
        # init_cond  -    почтакові умови;                      type(init_cond) == lambda-function or Piecewise
        # t0         -    початковий момент часу;               type(t0) == float
        # x0         -    початок відрізку розбиття;            type(x0) == float
        # step_t     -    крок по часу;                         type(step_t) == float
        # step_x     -    крок по відрізку;                     type(step_x) == float
        # T          -    кінцевий момент часу;                 type(T) == float
        # L          -    кінець відрізку розбиття;             type(L) == float
        # degree     -    степінь апроксимаційного поліному;    type(degree) == int
        # n_dots     -    кількість точок для апроксимації;     type(n_dots) == int
        
        self.k = k
        self.a2 = a2
        
        self.U_left = U_left
        self.U_right = U_right
        self.B_0 = init_cond
        
        self.t_part = np.arange(t0, T+step_t, step_t)
        self.x_part = np.arange(x0, L+step_x, step_x)
        
        self.degree = degree
        self.n_dots = n_dots
        
        self.Z_global = []
        self.Z_prime_global = []
        self.UQ_global = []
        
    def get_prime(self, Z):
        if type(Z) == Piecewise:
            prime = Z.diff(var_x)
            return lambdify(var_x, prime)
        else:
            prime = Z(var_x).diff(var_x)
            return lambdify(var_x, prime)
        
    def dependency_in_area(self, x, t, Z, Z_prime):
        
        # Z       - Функція апроксимаційного поліному;                type(Z) == lambda-function
        # Z_prime - Похідна від функції апроксимаційного поліному;    type(Z_prime) == lambda-function
        
        b = np.sqrt(self.a2*t/2)
        
        A_local = np.array([
            [          np.cosh(x/b),  -b/self.k*np.sinh(x/b)],
            [-self.k/b*np.sinh(x/b),   np.cosh(x/b)         ]
        ])
        
        B_local = np.array([Z(x), -self.k*Z_prime(x)])
        
        return A_local, B_local
    
    def build_full_system(self, t, Z, Z_prime):
        
        # Z       - масив функцій апроксимаційних поліномів;                 type(Z) == list(lambda-function)
        # Z_prime - масив похідних від функцій апроксимаційних поліномів;    type(Z_prime) == list(lambda-function)
        
        dim = 4*(self.x_part.shape[0]-1)
        
        A = np.zeros((dim, dim))
        B = np.zeros(dim)
        
        A[0, 0], B[0] = 1, self.U_left
        A[-1, -2], B[-1] = 1, self.U_right
        
        for i in range(dim//4):
            A_local, B_local = self.dependency_in_area(self.x_part[i+1], t, Z[i], Z_prime[i])
            
            A[1+4*i:3+4*i, 4*i:4*(i+1)] = np.block([-A_local, np.eye(2)])
            B[1+4*i:3+4*i] = B_local
            
        for i in range((dim//4)-1):
            A[3+4*i:5+4*i, 2+4*i:2+4*(i+1)] = np.block([np.eye(2), -np.eye(2)])
            
        return A, B
    
    def solve_full_system(self, A, B):
        return np.linalg.solve(A, B)
    
    def build_polynomial_approximation(self, t, UQ, Z, Z_prime):
        
        # Z       - масив функцій апроксимаційних поліномів;                 type(Z) == list(lambda-function)
        # Z_prime - масив похідних від функцій апроксимаційних поліномів;    type(Z_prime) == list(lambda-function)
        
        values = []
        area = []
        test = []
        
        for i in range(self.x_part.shape[0]-1):
            area.append(np.linspace(self.x_part[i], self.x_part[i+1], self.n_dots, endpoint=True))
            values.append([UQ[4*i]])
            
            if self.n_dots != 2:
                for dot in area[i][1:-1]:
                    A_local, B_local = self.dependency_in_area(dot, t, Z[i], Z_prime[i])
                    values[i].append((A_local @ np.vstack(UQ[4*i:4*i+2]) + np.vstack(B_local))[0][0])
            
            values[i].append(UQ[4*i+2])
                
        area = np.array(area)
        values = np.array(values)
        
        polynoms = []
        for i in range(area.shape[0]):
            coeffs = np.polyfit(area[i], values[i], self.degree)
            f = np.poly1d(-coeffs.x)
            
            polynoms.append(Poly(f(var_x)))
        
        return polynoms
    
    def find_partial_solution(self, t, polynoms):
        b2 = self.a2*t/2
        b = np.sqrt(b2)
        
        c = []
        for i in range(self.degree+1):
            c.append(symbols('c_{}'.format(i)))
            
        expr = c[self.degree]*var_x**self.degree
        for i in range(self.degree-1, 0, -1):
            expr += c[i]*var_x**i
            
        if self.degree != 0:
            expr += c[0]
            
        f = Poly(expr, var_x)
        
        coeffs = []
        for g in polynoms:
            coeff = list(solve_undetermined_coeffs(Eq((b2*(f.diff(var_x).diff(var_x)) - f).as_expr(), g.as_expr()), c, var_x).values())
            coeff.reverse()
            coeffs.append(coeff)
        
        coeffs = np.array(coeffs)
        
        Z = []
        for i in range(coeffs.shape[0]):
            expr = coeffs[i][0]*var_x**self.degree
            for j in range(1, coeffs.shape[1]-2):
                expr += coeffs[i][j]*var_x**(self.degree-j)
            expr += coeffs[i][-2]*(var_x - b*sinh(var_x/b)) + coeffs[i][-1]*(1-cosh(var_x/b))
            
            Z.append(lambdify(var_x, expr, modules=sympy))
            
        return Z
    
    def ode_solve(self):
        dt = self.t_part[1]
        Z = []
        Z_prime = []
        
        if type(self.B_0) == Piecewise:
            B_0 = lambdify(var_x, self.B_0)
        
        for i in range(self.x_part.shape[0]-1):
            Z.append(B_0)
            Z_prime.append(self.get_prime(self.B_0))
            
        for dt in self.t_part[1:]:
            self.Z_global.append(Z)
            self.Z_prime_global.append(Z_prime)

            A, B = self.build_full_system(dt, Z, Z_prime)
            UQ = self.solve_full_system(A, B)

            self.UQ_global.append(UQ)

            x, y = self.build_polynomial_approximation(dt, UQ, Z, Z_prime)

            Z = self.find_partial_solution(dt, polynoms)
            for i in range(len(Z)):
                Z_prime[i] = self.get_prime(Z[i])

In [ ]:
k = 10
a2 = 1
U_left = 0
U_right = 0
init_cond = Piecewise((var_x, (var_x>=0)&(var_x<=pi/2)), (pi-var_x, (var_x>pi/2)&(var_x<=pi)))
t0 = 0
x0 = 0
step_t = 0.025
step_x = np.pi/40
T = 3
L = np.pi
degree = 3
n_dots = 1000

In [ ]:
ode = OdeHalfAnalyticalSolve(k, a2, U_left, U_right, init_cond, t0, x0, step_t, step_x, T, L, degree, n_dots)
ode.ode_solve()
UQ = np.array(ode.UQ_global)

In [ ]:
U = []
for i in range(t_part.shape[0]-1):
    U.append([])
    for j in range(x_part.shape[0]-1):
        U[i].append(UQ[i][4*j])
    U[i].append(UQ[i][-2])
        
U = np.array(U)
U.shape

In [ ]:
for i in range(t_part.shape[0]-1):
    plt.figure(figsize=(14,6))
    plt.plot(np.arange(x0, L+step_x, step_x), U[i])
    plt.grid(True)
    plt.show()